# 🚀 PostgreSQL Bulk Upload from CSV Data

This notebook uses Python, `pandas`, and `psycopg2` to efficiently load large amounts of CSV data into a PostgreSQL database using the fast `COPY FROM` method. The script is now adapted to clean and transform the raw data into the specific data types required by your target schema.

## Prerequisites

Make sure you have installed the required libraries:

```bash
!pip install pandas psycopg2-binary dotenv
```

### Database and Data Setup

**1. PostgreSQL Table:** Ensure your `cars` table exists and matches the detailed schema you provided.

**2. Credentials:** Update the `DB_NAME`, `DB_USER`, and `DB_PASS` variables in the code block below, or set them securely in a `.env` file.

**3. Data:** Replace the sample `csv_data_string` with the content of your CSV file.

In [20]:
import pandas as pd
import psycopg2
from io import StringIO
import os
from dotenv import load_dotenv
import re

# Load environment variables (optional, for security)
load_dotenv()

# --- Database Configuration ---
DB_NAME = os.getenv("DB_NAME", "car_prediction") # <-- CHANGE THIS
DB_USER = os.getenv("DB_USER", "bpr_user")       # <-- CHANGE THIS
DB_PASS = os.getenv("DB_PASS", "postgres")   # <-- CHANGE THIS
DB_HOST = os.getenv("DB_HOST", "localhost")
DB_PORT = os.getenv("DB_PORT", "5432")

TABLE_NAME = 'cars'

# --- Load CSV Data ---
df = pd.read_csv('book1.csv')
print(f"✅ Loaded {len(df)} rows from book1.csv")
print(f"Columns in CSV: {list(df.columns)}")
df.head()

✅ Loaded 29083 rows from book1.csv
Columns in CSV: ['url', 'brand', 'model', 'variant', 'title', 'price', 'description', 'details_model_year', 'details_first_registration', 'details_mileage_km', 'details_fuel_type', 'details_range_km', 'details_battery_capacity_kwh', 'details_energy_consumption', 'details_home_charging_ac', 'details_fast_charging_dc', 'details_charging_time_dc_10_80_pct', 'details_periodic_tax', 'details_power_hp_nm', 'details_acceleration_0_100', 'details_top_speed', 'details_towing_capacity', 'details_color', 'model_new_price', 'model_category', 'model_body_type', 'model_trunk_size', 'model_weight_kg', 'model_width_cm', 'model_length_cm', 'model_height_cm', 'model_load_capacity_kg', 'model_max_towing_with_brake', 'model_drive_type', 'model_abs_brakes', 'model_esp', 'model_airbags', 'model_doors', 'equipment', 'details_fuel_consumption', 'details_co2_emission', 'details_euro_norm', 'details_transmission_type', 'details_number_of_gears', 'model_cylinders', 'model_tank_

,url,brand,model,variant,title,price,description,details_model_year,details_first_registration,details_mileage_km,...,model_doors,equipment,details_fuel_consumption,details_co2_emission,details_euro_norm,details_transmission_type,details_number_of_gears,model_cylinders,model_tank_capacity_l,details_production_year
0,https://www.bilbasen.dk/brugt/bil/ford/explore...,Ford,Explorer,82 Premium Extended Range 5d,Ford Explorer82 Premium Extended Range 5d,279.800 kr.,DER KAN TILKØBES 5 ÅRS FRAGUS GARANTI INDTIL 5...,2024.0,9/2024,31.000 km,...,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.bilbasen.dk/brugt/bil/ford/kuga/25...,Ford,Kuga,"2,5 PHEV ST-Line X CVT 5d","Ford Kuga2,5 PHEV ST-Line X CVT 5d",389.900 kr.,🚗 BILENS UDSTYR: ✅ Vinterpakke (Varme i rat + ...,2024.0,7/2024,16.686 km,...,5,ABS-bremserAutomatgear,"(NEDC) 43,1 km/l",22 g/km,6,Automatisk,1,4,43 l,NaN
2,https://www.bilbasen.dk/brugt/bil/toyota/aygo/...,Toyota,Aygo,"1,0 VVT-i x-press x-shift 5d","Toyota Aygo1,0 VVT-i x-press x-shift 5d",119.900 kr.,FLERE BILLEDER PÅ VEJ - 1 EJERS UDSTYRSVOGN - ...,2021.0,3/2021,26.000 km,...,5,NaN,"(WLTP) 19,2 km/l",118 g/km,6,Automatisk,5,3,35 l,NaN
3,https://www.bilbasen.dk/brugt/bil/polestar/2/l...,Polestar,2,Long Range 5d,Polestar 2Long Range 5d,209.900 kr.,"🚗 Drømmebilen er tættere på, end du tror! 🚗 💰 ...",2023.0,4/2023,49.000 km,...,5,ABS-bremserAutomatgear,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.bilbasen.dk/brugt/bil/seat/ibiza/1...,Seat,Ibiza,"1,0 MPi 80 Style 5d","Seat Ibiza1,0 MPi 80 Style 5d",159.900 kr.,Er du på udkig efter en pålidelig og stilfuld ...,2024.0,12/2023,22.000 km,...,5,Auto. nødbremseMultifunktionsratAutomatisk nød...,"(WLTP) 18,8 km/l",119 g/km,-,Manuel,5,3,40 l,11/2023


In [8]:
# Debug: Check raw data types and values
print("Raw data inspection:")
print(f"Total rows: {len(df)}")
print(f"\nFirst few 'price' values (raw):")
print(df['price'].head(10))
print(f"\nPrice data type: {df['price'].dtype}")
print(f"\nFirst few 'brand' values:")
print(df['brand'].head(10))
print(f"\nFirst few 'model' values:")
print(df['model'].head(10))

Raw data inspection:
Total rows: 29083

First few 'price' values (raw):
0    279.800 kr.
1    389.900 kr.
2    119.900 kr.
3    209.900 kr.
4    159.900 kr.
5    184.800 kr.
6     45.000 kr.
7     79.900 kr.
8    162.900 kr.
9    194.500 kr.
Name: price, dtype: object

Price data type: object

First few 'brand' values:
0        Ford
1        Ford
2      Toyota
3    Polestar
4        Seat
5          VW
6        Opel
7         Kia
8          VW
9       Skoda
Name: brand, dtype: object

First few 'model' values:
0    Explorer
1        Kuga
2        Aygo
3           2
4       Ibiza
5        ID.3
6        Karl
7        Ceed
8        ID.3
9       Enyaq
Name: model, dtype: object


In [9]:
# Analyze unique values in categorical columns
print("=" * 60)
print("FUEL TYPE Analysis")
print("=" * 60)
fuel_values = df['details_fuel_type'].value_counts()
print(f"Total unique fuel types: {len(fuel_values)}")
print(fuel_values)

print("\n" + "=" * 60)
print("BODY TYPE Analysis")
print("=" * 60)
body_values = df['model_body_type'].value_counts()
print(f"Total unique body types: {len(body_values)}")
print(body_values)

print("\n" + "=" * 60)
print("TRANSMISSION Analysis")
print("=" * 60)
trans_values = df['details_transmission_type'].value_counts()
print(f"Total unique transmission types: {len(trans_values)}")
print(trans_values)

print("\n" + "=" * 60)
print("DRIVE TYPE Analysis")
print("=" * 60)
drive_values = df['model_drive_type'].value_counts()
print(f"Total unique drive types: {len(drive_values)}")
print(drive_values)

FUEL TYPE Analysis
Total unique fuel types: 7
details_fuel_type
El                       11298
Benzin                   10998
Diesel                    3898
Plug-in hybrid Benzin     1586
Hybrid (Benzin + El)       447
Plug-in hybrid Diesel      141
Hybrid (Diesel + El)         1
Name: count, dtype: int64

BODY TYPE Analysis
Total unique body types: 13
model_body_type
Halvkombi    7588
CUV          7417
St.car       3381
SUV          2880
Mikro        2778
Sedan        1514
MPV          1383
Cabriolet     855
Coupe         448
-              98
Minibus        21
4x4             4
Kassevogn       2
Name: count, dtype: int64

TRANSMISSION Analysis
Total unique transmission types: 3
details_transmission_type
Manuel        9324
Automatisk    7743
-                4
Name: count, dtype: int64

DRIVE TYPE Analysis
Total unique drive types: 4
model_drive_type
Forhjulstræk     17267
Baghjulstræk      7007
Firehjulstræk     3893
-                  202
Name: count, dtype: int64


## 1. Data Cleaning and Transformation

This section maps the Danish column headers to your robust PostgreSQL column names and performs cleaning and type conversions for numeric, boolean, and textual data.

In [21]:
# --- Column Mapping (CSV to Database) ---
column_map = {
    'url': 'url',
    'brand': 'brand',
    'model': 'model',
    'variant': 'variant',
    'title': 'title',
    'price': 'price_raw',
    'description': 'description',
    'details_model_year': 'model_year',
    'details_first_registration': 'first_registration',
    'details_mileage_km': 'mileage_raw',
    'details_fuel_type': 'fuel_type',
    'details_range_km': 'range_km',
    'details_battery_capacity_kwh': 'battery_capacity',
    'details_energy_consumption': 'energy_consumption',
    'details_home_charging_ac': 'home_charging_ac',
    'details_fast_charging_dc': 'fast_charging_dc',
    'details_charging_time_dc_10_80_pct': 'charging_time_dc',
    'details_periodic_tax': 'periodic_tax',
    'details_power_hp_nm': 'power_raw',
    'details_acceleration_0_100': 'acceleration',
    'details_top_speed': 'top_speed',
    'details_towing_capacity': 'towing_capacity',
    'details_color': 'color',
    'model_new_price': 'new_price_raw',
    'model_category': 'category',
    'model_body_type': 'body_type',
    'model_trunk_size': 'trunk_size',
    'model_weight_kg': 'weight',
    'model_width_cm': 'width',
    'model_length_cm': 'length',
    'model_height_cm': 'height',
    'model_load_capacity_kg': 'load_capacity',
    'model_max_towing_with_brake': 'max_towing_weight',
    'model_drive_type': 'drive_type',
    'model_abs_brakes': 'abs_brakes',
    'model_esp': 'esp',
    'model_airbags': 'airbags',
    'model_doors': 'doors',
    'equipment': 'equipment',
    'details_fuel_consumption': 'fuel_consumption',
    'details_co2_emission': 'co2_emission',
    'details_euro_norm': 'euro_norm',
    'details_transmission_type': 'transmission',
    'details_number_of_gears': 'gear_count',
    'model_cylinders': 'cylinders',
    'model_tank_capacity_l': 'tank_capacity',
    'details_production_year': 'production_date'
}

# Rename columns
df = df.rename(columns=column_map)

import numpy as np

# Helper to safely cast numeric to Int64 (rounding)
def to_int64(series):
    ser = pd.to_numeric(series, errors='coerce')
    ser = ser.apply(lambda x: int(round(x)) if pd.notna(x) else pd.NA)
    return ser.astype('Int64')

# Clean price - remove "kr." and dots (thousands separator)
print("Cleaning price...")
print(f"Sample raw prices: {df['price_raw'].head(3).tolist()}")
df['price_raw'] = df['price_raw'].astype(str).str.replace(' kr.', '', regex=False).str.replace('.', '', regex=False).str.replace(',', '.', regex=False).str.strip()
print(f"Sample cleaned prices: {df['price_raw'].head(3).tolist()}")
df['price'] = pd.to_numeric(df['price_raw'], errors='coerce')
print(f"Sample numeric prices: {df['price'].head(3).tolist()}")
print(f"Non-null prices: {df['price'].notna().sum()} / {len(df)}")

# Clean new_price
df['new_price_raw'] = df['new_price_raw'].astype(str).str.replace(' kr.', '', regex=False).str.replace('.', '', regex=False).str.replace(',', '.', regex=False).str.strip()
df['new_price'] = pd.to_numeric(df['new_price_raw'], errors='coerce')

# Clean mileage
df['mileage_raw'] = df['mileage_raw'].astype(str).str.replace(' km', '', regex=False).str.replace('.', '', regex=False).str.strip()
df['mileage'] = to_int64(df['mileage_raw'])

# Year
df['model_year'] = to_int64(df['model_year'])
df['year'] = df['model_year']

# Other numeric columns
df['range_km'] = to_int64(df['range_km'])
df['battery_capacity'] = pd.to_numeric(df['battery_capacity'], errors='coerce').round(1)
df['energy_consumption'] = to_int64(df['energy_consumption'])
df['acceleration'] = pd.to_numeric(df['acceleration'], errors='coerce').round(1)
df['top_speed'] = to_int64(df['top_speed'])
df['towing_capacity'] = to_int64(df['towing_capacity'])
df['trunk_size'] = to_int64(df['trunk_size'])
df['weight'] = to_int64(df['weight'])
df['width'] = to_int64(df['width'])
df['length'] = to_int64(df['length'])
df['height'] = to_int64(df['height'])
df['load_capacity'] = to_int64(df['load_capacity'])
df['max_towing_weight'] = to_int64(df['max_towing_weight'])
df['gear_count'] = to_int64(df['gear_count'])
df['cylinders'] = to_int64(df['cylinders'])
df['tank_capacity'] = to_int64(df['tank_capacity'])
df['airbags'] = to_int64(df['airbags'])
df['doors'] = to_int64(df['doors'])

# Extract horsepower and torque from power_raw (format: "XXX hk/YYY nm")
power_split = df['power_raw'].astype(str).str.extract(r'(\d+)\s*hk[/\s]*(\d+)?\s*nm?', expand=True)
df['horsepower'] = to_int64(power_split[0])
df['torque_nm'] = to_int64(power_split[1])

# Boolean conversions
for col in ['abs_brakes', 'esp']:
    df[col] = df[col].map({'True': True, 'False': False, 'Ja': True, 'Nej': False, 
                            'ja': True, 'nej': False, True: True, False: False})

# Text columns - ensure strings and clean
text_cols = ['brand', 'model', 'variant', 'title', 'description', 'fuel_type', 
             'transmission', 'color', 'category', 'body_type', 'drive_type', 'equipment',
             'fuel_consumption', 'co2_emission', 'euro_norm', 'periodic_tax',
             'home_charging_ac', 'fast_charging_dc', 'charging_time_dc',
             'first_registration', 'production_date']

for col in text_cols:
    df[col] = df[col].fillna('').astype(str).str.strip()
    df[col] = df[col].replace(['', 'nan', 'None'], None)

# ========== STANDARDIZE CATEGORICAL VALUES (Danish + English → 7 FUEL TYPES) ==========
print("\n" + "=" * 60)
print("STANDARDIZING CATEGORICAL VALUES TO 7 FUEL TYPES")
print("=" * 60)

print(f"\nBEFORE standardization - Unique fuel types: {df['fuel_type'].nunique()}")
print(df['fuel_type'].value_counts())

# Define exactly 7 fuel type categories
def standardize_fuel_type(fuel):
    if pd.isna(fuel) or fuel == '' or fuel is None:
        return None
    
    # Normalize: lowercase and remove all hyphens/spaces for comparison
    fuel_normalized = str(fuel).strip().lower().replace('-', '').replace(' ', '')
    fuel_original = str(fuel).strip().lower()
    
    # 1. Electricity (El, Electric, Electricity)
    if fuel_normalized in ['el', 'electricity', 'electric', 'elektricitet']:
        return 'Electricity'
    
    # 2. Petrol (Benzin, Petrol, Gasoline) - NOT hybrid
    if fuel_normalized in ['benzin', 'petrol', 'gasoline'] and 'hybrid' not in fuel_original:
        return 'Petrol'
    
    # 3. Diesel - NOT hybrid
    if fuel_normalized == 'diesel' and 'hybrid' not in fuel_original:
        return 'Diesel'
    
    # 4 & 5. Plug-in Hybrid (check first before regular hybrid)
    if ('plugin' in fuel_normalized or 'pluginhybrid' in fuel_normalized):
        if 'diesel' in fuel_original:
            return 'Plug-in Hybrid - Diesel'
        else:
            return 'Plug-in Hybrid - Petrol'
    
    # 6 & 7. Regular Hybrid (non plug-in)
    if 'hybrid' in fuel_original:
        if 'diesel' in fuel_original:
            return 'Hybrid - Diesel'
        else:
            return 'Hybrid - Petrol'
    
    # Default: if not recognized, log it and return as-is for debugging
    print(f"⚠️ Unrecognized fuel type: '{fuel}' -> normalized: '{fuel_normalized}'")
    return str(fuel).strip()

df['fuel_type'] = df['fuel_type'].apply(standardize_fuel_type)

# FINAL CONSOLIDATION - Map any remaining variations to the 7 standard types
final_fuel_mapping = {
    # Electricity variations
    'Electricity': 'Electricity',
    'Electric': 'Electricity',
    'El': 'Electricity',
    
    # Petrol variations
    'Petrol': 'Petrol',
    'Benzin': 'Petrol',
    
    # Diesel variations
    'Diesel': 'Diesel',
    
    # Hybrid - Petrol variations (default for any unspecified "Hybrid")
    'Hybrid': 'Hybrid - Petrol',
    'Hybrid - Petrol': 'Hybrid - Petrol',
    'Hybrid (Petrol)': 'Hybrid - Petrol',
    
    # Hybrid - Diesel variations
    'Hybrid - Diesel': 'Hybrid - Diesel',
    'Hybrid (Diesel)': 'Hybrid - Diesel',
    
    # Plug-in Hybrid - Petrol variations (default for any unspecified "Plug-in Hybrid")
    'Plug-in Hybrid': 'Plug-in Hybrid - Petrol',
    'Plugin Hybrid': 'Plug-in Hybrid - Petrol',
    'Plug-in-Hybrid': 'Plug-in Hybrid - Petrol',
    'Plugin-Hybrid': 'Plug-in Hybrid - Petrol',
    'Plug-In Hybrid': 'Plug-in Hybrid - Petrol',
    'Plug-in Hybrid - Petrol': 'Plug-in Hybrid - Petrol',
    'Plugin Hybrid - Petrol': 'Plug-in Hybrid - Petrol',
    
    # Plug-in Hybrid - Diesel variations
    'Plug-in Hybrid - Diesel': 'Plug-in Hybrid - Diesel',
    'Plugin Hybrid - Diesel': 'Plug-in Hybrid - Diesel',
    'Plug-in-Hybrid - Diesel': 'Plug-in Hybrid - Diesel',
}

df['fuel_type'] = df['fuel_type'].map(final_fuel_mapping).fillna(df['fuel_type'])

print(f"\nAFTER final consolidation - Unique fuel types: {df['fuel_type'].nunique()}")
print(df['fuel_type'].value_counts())
print(f"\nFuel types present: {sorted(df['fuel_type'].dropna().unique())}")

# Verify we have exactly 7 or fewer types
unique_fuel_types = df['fuel_type'].dropna().nunique()
if unique_fuel_types > 7:
    print(f"\n⚠️ WARNING: Still have {unique_fuel_types} fuel types (expected 7)!")
    print("Unmapped fuel types:")
    print(df['fuel_type'].value_counts())
else:
    print(f"\n✅ SUCCESS: Consolidated to {unique_fuel_types} fuel types!")

# 2. BODY TYPE - Standardize (including Danish types from screenshot)
body_type_mapping = {
    # SUV
    'SUV': 'SUV',
    'Suv': 'SUV',
    'Cuv': 'SUV',
    'CUV': 'SUV',
    
    # Sedan
    'Sedan': 'Sedan',
    
    # Station Wagon / Stationcar / St.car
    'Stationcar': 'Station Wagon',
    'Station wagon': 'Station Wagon',
    'Station Wagon': 'Station Wagon',
    'Stasjonsvogn': 'Station Wagon',
    'St.car': 'Station Wagon',  # Added from screenshot
    
    # Hatchback / Halvkombi / Mikro
    'Hatchback': 'Hatchback',
    'Halvkombi': 'Hatchback',
    'Mikro': 'Hatchback',  # Added from screenshot (small city cars)
    
    # Coupe / Cabriolet
    'Coupé': 'Coupe',
    'Coupe': 'Coupe',
    'Cabriolet': 'Cabriolet',
    'Kabrio': 'Cabriolet',
    
    # Van / MPV / Kassevogn / Wagon
    'Minibus': 'Van',
    'Van': 'Van',
    'MPV': 'Van',
    'Varevogn': 'Van',
    'Kassevogn': 'Van',  # Added from screenshot
    'Wagon': 'Van',  # Added from screenshot
    
    # Pickup / 4x4
    'Pickup': 'Pickup',
    'Pick-up': 'Pickup',
    '4x4': 'Pickup',  # Added from screenshot (typically pickup trucks)
}

df['body_type'] = df['body_type'].map(body_type_mapping).fillna(df['body_type'])
print(f"\nBody types after standardization: {df['body_type'].nunique()} unique types")
print(df['body_type'].value_counts())

# 3. TRANSMISSION - Standardize AND set Automatic for electric cars
transmission_mapping = {
    # Automatic
    'Automatgear': 'Automatic',
    'Automatic': 'Automatic',
    'Automatisk': 'Automatic',
    'Auto': 'Automatic',
    'Automat': 'Automatic',
    
    # Manual
    'Manuel': 'Manual',
    'Manual': 'Manual',
    'Manuell': 'Manual',
    'Manuelt': 'Manual',
}

df['transmission'] = df['transmission'].map(transmission_mapping).fillna(df['transmission'])

# Electric cars don't have traditional transmissions - set to Automatic
electric_mask = (df['fuel_type'] == 'Electricity')
df.loc[electric_mask & df['transmission'].isna(), 'transmission'] = 'Automatic'
df.loc[electric_mask & (df['transmission'] == '-'), 'transmission'] = 'Automatic'

print(f"\nTransmission types after standardization: {df['transmission'].nunique()} unique types")
print(df['transmission'].value_counts())
print(f"Electric cars with Automatic transmission: {df[electric_mask & (df['transmission'] == 'Automatic')].shape[0]}")

# 4. DRIVE TYPE - Standardize
drive_type_mapping = {
    # Front Wheel Drive
    'Forhjulstræk': 'Front-Wheel Drive',
    'Front-wheel drive': 'Front-Wheel Drive',
    'FWD': 'Front-Wheel Drive',
    'Forhjulsdrift': 'Front-Wheel Drive',
    
    # Rear Wheel Drive
    'Baghjulstræk': 'Rear-Wheel Drive',
    'Rear-wheel drive': 'Rear-Wheel Drive',
    'RWD': 'Rear-Wheel Drive',
    'Bakhjulsdrift': 'Rear-Wheel Drive',
    
    # All Wheel Drive / 4WD
    'Firehjulstræk': 'All-Wheel Drive',
    'All-wheel drive': 'All-Wheel Drive',
    'AWD': 'All-Wheel Drive',
    '4WD': 'All-Wheel Drive',
    'Firehjulsdrift': 'All-Wheel Drive',
    '4x4': 'All-Wheel Drive',
}

df['drive_type'] = df['drive_type'].map(drive_type_mapping).fillna(df['drive_type'])
print(f"\nDrive types after standardization: {df['drive_type'].nunique()} unique types")
print(df['drive_type'].value_counts())

print("\n" + "=" * 60)
print("✅ Categorical standardization complete!")
print("=" * 60)

# Check filtering condition BEFORE filtering
print(f"\nPre-filter check:")
print(f"  Non-null brand: {df['brand'].notna().sum()}")
print(f"  Non-null model: {df['model'].notna().sum()}")
print(f"  Non-null price: {df['price'].notna().sum()}")
print(f"  Price > 0: {(df['price'] > 0).sum()}")

# Drop rows missing essential fields
df = df[(df['brand'].notna()) & (df['model'].notna()) & (df['price'].notna()) & (df['price'] > 0)]

print(f"\nAfter filtering: {len(df)} rows")

# Add missing required columns with defaults
if 'seats' not in df.columns:
    df['seats'] = 5

if 'engine_size' not in df.columns:
    def extract_engine_size(text):
        if pd.isna(text) or text == '' or text is None:
            return None
        m = re.search(r'(\d)[\.,](\d)', str(text))
        if m:
            try:
                return float(f"{m.group(1)}.{m.group(2)}")
            except:
                return None
        return None
    
    df['engine_size'] = df['variant'].apply(extract_engine_size)

df['location'] = None
df['dealer_name'] = None
df['source_url'] = df['url']

# Remove duplicate URLs
df = df.drop_duplicates(subset=['url'])

# Final column order matching database schema exactly
TARGET_COLUMNS = [
    'url', 'brand', 'model', 'variant', 'title', 'description',
    'price', 'new_price',
    'model_year', 'year', 'first_registration', 'production_date',
    'mileage', 'fuel_type', 'transmission', 'gear_count', 'cylinders',
    'horsepower', 'torque_nm', 'acceleration', 'top_speed',
    'range_km', 'battery_capacity', 'energy_consumption', 'home_charging_ac', 
    'fast_charging_dc', 'charging_time_dc',
    'fuel_consumption', 'co2_emission', 'euro_norm', 'tank_capacity',
    'body_type', 'weight', 'width', 'length', 'height', 'trunk_size',
    'load_capacity', 'towing_capacity', 'max_towing_weight',
    'drive_type', 'abs_brakes', 'esp', 'airbags',
    'doors', 'seats',
    'color', 'category', 'equipment', 'periodic_tax',
    'engine_size', 'source_url', 'location', 'dealer_name'
]

# Select final columns
df = df[TARGET_COLUMNS]

print(f"\n✅ Cleaned {len(df)} rows ready for upload")
print(f"\nSample of key data:")
print(df[['brand', 'model', 'price', 'year', 'mileage', 'horsepower', 'fuel_type', 'transmission']].head(10))
print(f"\nNull counts for important columns:")
print(df[['brand', 'model', 'price', 'year', 'mileage', 'fuel_type', 'horsepower', 'transmission']].isna().sum())
if len(df) > 0:
    print(f"\nPrice range: {df['price'].min():.0f} - {df['price'].max():.0f} kr")
    print(f"Year range: {df['year'].min():.0f} - {df['year'].max():.0f}")
    print(f"\nFINAL FUEL TYPE BREAKDOWN:")
    print(df['fuel_type'].value_counts())
    print(f"\nFINAL BODY TYPE BREAKDOWN:")
    print(df['body_type'].value_counts())

Cleaning price...
Sample raw prices: ['279.800 kr.', '389.900 kr.', '119.900 kr.']
Sample cleaned prices: ['279800', '389900', '119900']
Sample numeric prices: [279800.0, 389900.0, 119900.0]
Non-null prices: 28369 / 29083

STANDARDIZING CATEGORICAL VALUES TO 7 FUEL TYPES

BEFORE standardization - Unique fuel types: 7
fuel_type
El                       11298
Benzin                   10998
Diesel                    3898
Plug-in hybrid Benzin     1586
Hybrid (Benzin + El)       447
Plug-in hybrid Diesel      141
Hybrid (Diesel + El)         1
Name: count, dtype: int64

AFTER final consolidation - Unique fuel types: 7
fuel_type
Electricity                11298
Petrol                     10998
Diesel                      3898
Plug-in Hybrid - Petrol     1586
Hybrid - Petrol              447
Plug-in Hybrid - Diesel      141
Hybrid - Diesel                1
Name: count, dtype: int64

Fuel types present: ['Diesel', 'Electricity', 'Hybrid - Diesel', 'Hybrid - Petrol', 'Petrol', 'Plug-in Hybrid 

In [11]:
# Integrity & sanity checks before upload
import pandas as pd

issues = []

# Mandatory non-null fields
mandatory = ['brand','model','price']
for m in mandatory:
    missing = df[m].isna().sum() + (df[m] == '').sum()
    if missing > 0:
        issues.append(f"Missing values in mandatory field {m}: {missing}")

# Price non-negative
if (df['price'] < 0).any():
    issues.append("Negative prices detected")

# Year sanity
current_year = pd.Timestamp.now().year
invalid_years = df[~df['year'].isna() & ((df['year'] < 1900) | (df['year'] > current_year + 1))]
if len(invalid_years) > 0:
    issues.append(f"Year out of bounds rows: {len(invalid_years)}")

# Doors constraint (2-5) - FIX invalid values
bad_doors_before = df[df['doors'].notna() & ~df['doors'].between(2,5)]
if len(bad_doors_before) > 0:
    print(f"⚠ Fixing {len(bad_doors_before)} rows with doors outside 2-5 range")
    df.loc[df['doors'].notna() & ~df['doors'].between(2,5), 'doors'] = pd.NA

# Seats constraint (2-9) - FIX invalid values
bad_seats = df[df['seats'].notna() & ~df['seats'].between(2,9)]
if len(bad_seats) > 0:
    print(f"⚠ Fixing {len(bad_seats)} rows with seats outside 2-9 range")
    df.loc[df['seats'].notna() & ~df['seats'].between(2,9), 'seats'] = 5  # Default to 5

# Battery capacity plausible (0-200 kWh)
bad_bcap = df[df['battery_capacity'].notna() & ~df['battery_capacity'].between(0,200)]
if len(bad_bcap) > 0:
    print(f"⚠ Fixing {len(bad_bcap)} rows with battery capacity outside 0-200 kWh")
    df.loc[df['battery_capacity'].notna() & ~df['battery_capacity'].between(0,200), 'battery_capacity'] = pd.NA

# Acceleration plausible (0-50 s)
bad_acc = df[df['acceleration'].notna() & ~df['acceleration'].between(0,50)]
if len(bad_acc) > 0:
    print(f"⚠ Fixing {len(bad_acc)} rows with acceleration outside 0-50s")
    df.loc[df['acceleration'].notna() & ~df['acceleration'].between(0,50), 'acceleration'] = pd.NA

# Duplicate URL count
dup_urls = df['url'].duplicated().sum()
if dup_urls > 0:
    issues.append(f"Duplicate URLs in frame: {dup_urls}")

print(f"\n✅ Rows prepared: {len(df)} | Columns: {len(df.columns)}")
print("First 5 columns null counts:")
print(df[df.columns[:5]].isna().sum())

if issues:
    print("\n⚠ Issues detected (will proceed, but review if upload fails):")
    for i in issues:
        print(" -", i)
else:
    print("\n✅ All integrity checks passed!")

# Show a small sample of important numeric fields
print("\nSample numeric snapshot:")
print(df[['price','mileage','horsepower','battery_capacity','acceleration']].head())

⚠ Fixing 11 rows with doors outside 2-5 range

✅ Rows prepared: 28369 | Columns: 54
First 5 columns null counts:
url        0
brand      0
model      0
variant    0
title      0
dtype: int64

✅ All integrity checks passed!

Sample numeric snapshot:
      price  mileage  horsepower  battery_capacity  acceleration
0  279800.0    31000         286               NaN           NaN
1  389900.0    16686        <NA>               NaN           NaN
2  119900.0    26000          72               NaN           NaN
3  209900.0    49000         231               NaN           NaN
4  159900.0    22000          80               NaN           NaN


## 2. PostgreSQL Bulk Upload (`copy_from`)

The `psycopg2.copy_from` function uses PostgreSQL's native bulk load protocol, ensuring fast insertion into the defined schema.

In [12]:
# Check actual database column names
conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASS,
    host=DB_HOST,
    port=DB_PORT
)
cur = conn.cursor()
cur.execute("""
    SELECT column_name, data_type 
    FROM information_schema.columns 
    WHERE table_name = 'cars' 
    ORDER BY ordinal_position
""")
db_columns = cur.fetchall()
conn.close()

print("Database columns:")
for col_name, col_type in db_columns:
    print(f"  {col_name}: {col_type}")

Database columns:
  id: uuid
  url: text
  brand: character varying
  model: character varying
  variant: character varying
  title: character varying
  description: text
  price: numeric
  new_price: numeric
  model_year: integer
  year: integer
  first_registration: character varying
  production_date: character varying
  mileage: integer
  fuel_type: character varying
  transmission: character varying
  gear_count: integer
  cylinders: integer
  horsepower: integer
  torque_nm: integer
  acceleration: numeric
  top_speed: integer
  range_km: integer
  battery_capacity: numeric
  energy_consumption: integer
  home_charging_ac: character varying
  fast_charging_dc: character varying
  charging_time_dc: character varying
  fuel_consumption: character varying
  co2_emission: character varying
  euro_norm: character varying
  tank_capacity: integer
  body_type: character varying
  weight: integer
  width: integer
  length: integer
  height: integer
  trunk_size: integer
  load_capacity: 

In [13]:
def upload_data_to_postgres(df: pd.DataFrame, table_name: str, target_columns: list):
    """
    Connects to PostgreSQL and uses COPY ... WITH (FORMAT CSV) for reliable bulk insertion.
    """
    conn = None
    try:
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASS,
            host=DB_HOST,
            port=DB_PORT
        )
        cur = conn.cursor()
        print(f"Connected to PostgreSQL database: {DB_NAME}")

        buffer = StringIO()
        df.to_csv(buffer, header=False, index=False, na_rep='')
        buffer.seek(0)
        column_list = ", ".join(target_columns)
        copy_sql = (
            f"COPY {table_name} ({column_list}) "
            "FROM STDIN WITH (FORMAT CSV, DELIMITER ',', QUOTE '\"', ESCAPE '\"', NULL '')"
        )
        cur.copy_expert(sql=copy_sql, file=buffer)

        conn.commit()
        print(f"\n✅ Success! Uploaded {len(df)} rows to table '{table_name}'.")

    except psycopg2.Error as e:
        print(f"\n❌ Database Error: {e}")
        if conn:
            conn.rollback()
    except Exception as e:
        print(f"\n❌ An unexpected error occurred: {e}")
    finally:
        if conn:
            conn.close()
            print("Database connection closed.")

#upload_data_to_postgres(df, TABLE_NAME, TARGET_COLUMNS)

In [18]:
def update_existing_records(df: pd.DataFrame, table_name: str, target_columns: list):
    """
    Updates existing records in PostgreSQL based on URL (unique identifier).
    Only updates records that already exist in the database.
    """
    conn = None
    try:
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASS,
            host=DB_HOST,
            port=DB_PORT
        )
        cur = conn.cursor()
        print(f"Connected to PostgreSQL database: {DB_NAME}")
        
        # Get all existing URLs from database
        cur.execute(f"SELECT url FROM {table_name}")
        existing_urls = set(row[0] for row in cur.fetchall())
        print(f"Found {len(existing_urls)} existing records in database")
        
        # Filter dataframe to only include existing URLs
        df_update = df[df['url'].isin(existing_urls)].copy()
        print(f"Found {len(df_update)} matching records to update")
        
        if len(df_update) == 0:
            print("No records to update.")
            return
        
        # Build UPDATE query
        # Exclude 'url' from the columns to update (it's the WHERE condition)
        update_cols = [col for col in target_columns if col != 'url']
        
        updated_count = 0
        failed_count = 0
        
        print("\nUpdating records...")
        for idx, row in df_update.iterrows():
            try:
                # Build SET clause
                set_clause = ", ".join([f"{col} = %s" for col in update_cols])
                update_sql = f"UPDATE {table_name} SET {set_clause} WHERE url = %s"
                
                # Prepare values (all update columns + url for WHERE clause)
                values = [row[col] if pd.notna(row[col]) else None for col in update_cols]
                values.append(row['url'])
                
                cur.execute(update_sql, values)
                updated_count += 1
                
                if updated_count % 100 == 0:
                    print(f"  Updated {updated_count} records...")
                    
            except Exception as e:
                failed_count += 1
                if failed_count <= 5:  # Only print first 5 errors
                    print(f"  ⚠️ Failed to update {row['url']}: {e}")
        
        conn.commit()
        print(f"\n✅ Success! Updated {updated_count} records.")
        if failed_count > 0:
            print(f"⚠️ Failed to update {failed_count} records.")

    except psycopg2.Error as e:
        print(f"\n❌ Database Error: {e}")
        if conn:
            conn.rollback()
    except Exception as e:
        print(f"\n❌ An unexpected error occurred: {e}")
    finally:
        if conn:
            conn.close()
            print("Database connection closed.")

# Update existing records based on URL
update_existing_records(df, TABLE_NAME, TARGET_COLUMNS)

Connected to PostgreSQL database: car_prediction
Found 28370 existing records in database
Found 28369 matching records to update

Updating records...
Found 28370 existing records in database
Found 28369 matching records to update

Updating records...
  Updated 100 records...
  Updated 100 records...
  Updated 200 records...
  Updated 200 records...
  Updated 300 records...
  Updated 300 records...
  Updated 400 records...
  Updated 400 records...
  Updated 500 records...
  Updated 500 records...
  Updated 600 records...
  Updated 600 records...
  Updated 700 records...
  Updated 700 records...
  Updated 800 records...
  Updated 800 records...
  Updated 900 records...
  Updated 900 records...
  Updated 1000 records...
  Updated 1000 records...
  Updated 1100 records...
  Updated 1100 records...
  Updated 1200 records...
  Updated 1200 records...
  Updated 1300 records...
  Updated 1300 records...
  Updated 1400 records...
  Updated 1400 records...
  Updated 1500 records...
  Updated 150

KeyboardInterrupt: 

In [23]:
# --- OPTIMIZED BULK UPDATE - ONLY UPDATE CHANGED RECORDS ---
# This approach only updates records where data has actually changed

def bulk_update_changed_records_only(df: pd.DataFrame, table_name: str, target_columns: list):
    """
    Fast bulk update that only modifies records where data has changed.
    This saves time by skipping records that are already correct.
    """
    conn = None
    try:
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASS,
            host=DB_HOST,
            port=DB_PORT
        )
        cur = conn.cursor()
        print(f"Connected to PostgreSQL database: {DB_NAME}")
        
        # Fetch existing records that match our URLs
        url_list = df['url'].tolist()
        placeholders = ','.join(['%s'] * len(url_list))
        
        # Get current data from database for comparison
        fetch_sql = f"SELECT {', '.join(target_columns)} FROM {table_name} WHERE url IN ({placeholders})"
        cur.execute(fetch_sql, url_list)
        
        db_records = cur.fetchall()
        print(f"Found {len(db_records)} existing records in database")
        
        if len(db_records) == 0:
            print("No matching records to update.")
            return
        
        # Convert database records to DataFrame for comparison - handle types properly
        df_db = pd.DataFrame(db_records, columns=target_columns)
        
        # Prepare new data
        df_update = df[df['url'].isin(df_db['url'])].copy()
        
        # Fix doors constraint - ensure doors is between 2-5
        if 'doors' in df_update.columns:
            df_update.loc[df_update['doors'] > 5, 'doors'] = 5
            df_update.loc[df_update['doors'] < 2, 'doors'] = 4
        
        # Normalize both dataframes for comparison - handle nullable integers properly
        def normalize_df(dataframe):
            df_norm = dataframe.copy()
            for col in df_norm.columns:
                # Convert Int64 (nullable int) to regular object type for comparison
                if pd.api.types.is_integer_dtype(df_norm[col]) or str(df_norm[col].dtype) == 'Int64':
                    df_norm[col] = df_norm[col].apply(lambda x: str(int(x)) if pd.notna(x) and x != '' else '')
                elif pd.api.types.is_float_dtype(df_norm[col]):
                    df_norm[col] = df_norm[col].apply(lambda x: str(float(x)) if pd.notna(x) and x != '' else '')
                elif pd.api.types.is_bool_dtype(df_norm[col]):
                    df_norm[col] = df_norm[col].apply(lambda x: str(bool(x)) if pd.notna(x) and x != '' else '')
                else:
                    df_norm[col] = df_norm[col].fillna('').astype(str)
            return df_norm
        
        print("Comparing database records with new data...")
        df_db_norm = normalize_df(df_db)
        df_update_norm = normalize_df(df_update)
        
        # Set URL as index for easy comparison
        df_db_norm = df_db_norm.set_index('url')
        df_update_norm = df_update_norm.set_index('url')
        
        # Find records that have changed
        changed_urls = []
        comparison_cols = [col for col in target_columns if col != 'url']
        
        for url in df_update_norm.index:
            if url in df_db_norm.index:
                # Compare all columns except url
                db_row = df_db_norm.loc[url, comparison_cols]
                new_row = df_update_norm.loc[url, comparison_cols]
                
                if not db_row.equals(new_row):
                    changed_urls.append(url)
        
        print(f"Found {len(changed_urls)} records with changes (out of {len(df_update)} total)")
        
        if len(changed_urls) == 0:
            print("✅ All records are already up to date!")
            return
        
        # Filter to only changed records
        df_changed = df_update[df_update['url'].isin(changed_urls)].copy()
        
        # Create temporary table
        temp_table = "temp_update_cars"
        col_definitions = [f"{col} TEXT" for col in target_columns]
        
        create_temp_sql = f"CREATE TEMP TABLE {temp_table} ({', '.join(col_definitions)})"
        cur.execute(create_temp_sql)
        print(f"Created temporary table for {len(df_changed)} changed records")
        
        # Load changed data into temp table
        buffer = StringIO()
        df_changed[target_columns].to_csv(buffer, index=False, header=False, sep='|')
        buffer.seek(0)
        
        copy_sql = f"COPY {temp_table} ({', '.join(target_columns)}) FROM STDIN WITH (FORMAT CSV, DELIMITER '|', NULL '')"
        cur.copy_expert(copy_sql, buffer)
        print(f"Loaded {len(df_changed)} changed records into temp table")
        
        # Build UPDATE FROM query
        update_cols = [col for col in target_columns if col != 'url']
        set_clause = ", ".join([f"{col} = CAST({temp_table}.{col} AS {get_column_type(col)})" 
                                for col in update_cols])
        
        update_sql = f"""
        UPDATE {table_name}
        SET {set_clause}
        FROM {temp_table}
        WHERE {table_name}.url = {temp_table}.url
        """
        
        cur.execute(update_sql)
        updated_count = cur.rowcount
        conn.commit()
        
        print(f"\n✅ Success! Updated {updated_count} changed records (skipped {len(df_update) - updated_count} unchanged).")
        
        # Drop temp table
        cur.execute(f"DROP TABLE {temp_table}")
        
    except psycopg2.Error as e:
        print(f"\n❌ Database Error: {e}")
        if conn:
            conn.rollback()
    except Exception as e:
        print(f"\n❌ An unexpected error occurred: {e}")
        import traceback
        traceback.print_exc()
    finally:
        if conn:
            conn.close()
            print("Database connection closed.")


def get_column_type(col_name):
    """Map column names to PostgreSQL types for CAST operations"""
    int_cols = ['model_year', 'year', 'mileage', 'range_km', 'gear_count', 'cylinders',
                'horsepower', 'torque_nm', 'top_speed', 'weight', 'width', 'length', 
                'height', 'trunk_size', 'load_capacity', 'towing_capacity', 
                'max_towing_weight', 'airbags', 'doors', 'seats', 'tank_capacity', 'energy_consumption']
    
    float_cols = ['price', 'new_price', 'acceleration', 'battery_capacity', 'engine_size']
    
    bool_cols = ['abs_brakes', 'esp']
    
    if col_name in int_cols:
        return 'INTEGER'
    elif col_name in float_cols:
        return 'NUMERIC'
    elif col_name in bool_cols:
        return 'BOOLEAN'
    else:
        return 'TEXT'


# Execute the optimized update (only changed records)
from io import StringIO
bulk_update_changed_records_only(df, TABLE_NAME, TARGET_COLUMNS)

Connected to PostgreSQL database: car_prediction
Found 28369 existing records in database
Found 28369 existing records in database
Comparing database records with new data...
Comparing database records with new data...
Found 28369 records with changes (out of 28369 total)
Created temporary table for 28369 changed records
Found 28369 records with changes (out of 28369 total)
Created temporary table for 28369 changed records
Loaded 28369 changed records into temp table
Loaded 28369 changed records into temp table

✅ Success! Updated 28369 changed records (skipped 0 unchanged).
Database connection closed.

✅ Success! Updated 28369 changed records (skipped 0 unchanged).
Database connection closed.
